[Open in Colab](https://colab.research.google.com/github/LapTQ/handwritten_text_recognition/blob/main/demo.ipynb)

In [ ]:
!git clone https://github.com/LapTQ/handwritten_text_recognition.git
!pip install jiwer
%cd handwritten_text_recognition

In [ ]:
!python train.py --invert_color True